In [22]:
# Dependencies and Setup--provided in startercode
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key
print(weather_api_key)
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

6d2f47049cb380aa267858b2e66a720f


In [23]:
#Generate CITIES LIST
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

614

In [24]:
#Perform API Calls
#Perform a weather check on each city using a series of successive API calls.
# Include a print log of each city as it'sbeing processed (with the city number and city name).
base_url = f'http://api.openweathermap.org/data/2.5/weather?q='
#loop through cities to create data frame
lat = []
lng = []
weathermain = []
w_description = []
temp = []
vis = []
temp= []
temp_feels = []
temp_min = []
temp_max = []
pressure = []
humidity = []
sealvl = []
clouds = []
windspeed = []
name = []
for city in cities:
    response=requests.get(base_url + city + '&appid=' + weather_api_key).json()
    print(response)
    lat.append(response['coord']['lat'])
    lng.append(response['coord']['lon'])
    w_description.append(response['weather'][0]['description'])
    weathermain.append(response['weather'][0]['main'])
    temp.append(response['main']['temp'])
    temp_feels.append(response['main']['feels_like'])
    temp_min.append(response['main']['temp_min'])
    temp_max.append(response['main']['temp_max'])
    pressure.append(response['main']['pressure'])
    humidity.append(response['main']['humidity'])
    clouds.append(response['clouds']['all'])
    vis.append(response['visibility'])
    windspeed.append(response['wind']['speed'])
    name.append(response['name'])
    

{'coord': {'lon': -68.3, 'lat': -54.8}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 279.15, 'feels_like': 276.47, 'temp_min': 279.15, 'temp_max': 279.15, 'pressure': 991, 'humidity': 87}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 90}, 'clouds': {'all': 75}, 'dt': 1619484632, 'sys': {'type': 1, 'id': 8303, 'country': 'AR', 'sunrise': 1619437501, 'sunset': 1619471795}, 'timezone': -10800, 'id': 3833367, 'name': 'Ushuaia', 'cod': 200}
{'coord': {'lon': -97.8558, 'lat': 55.7435}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 282.15, 'feels_like': 279.24, 'temp_min': 282.15, 'temp_max': 282.15, 'pressure': 999, 'humidity': 49}, 'visibility': 10000, 'wind': {'speed': 5.66, 'deg': 180, 'gust': 10.29}, 'clouds': {'all': 40}, 'dt': 1619484877, 'sys': {'type': 1, 'id': 935, 'country': 'CA', 'sunrise': 1619434727, 'sunset

KeyError: 'coord'

In [25]:
#create a data frame for these citites data
cities_dict = {
    "City" : name,
    "LAT": lat,
    'LON': lng,
    'Weather': weathermain,
    'Desription': w_description,
    'Temp': temp,
    'Temp Feels': temp_feels,
    'Temp Min' : temp_min,
    'Temp Max': temp_max,
    'Pressure': pressure,
    'Humidity': humidity,
    'Visibility': vis,
    'Clouds' : clouds,
    'Wind Speed': windspeed
}
cities_df = pd.DataFrame(cities_dict)
cities_df

,City,LAT,LON,Weather,Desription,Temp,Temp Feels,Temp Min,Temp Max,Pressure,Humidity,Visibility,Clouds,Wind Speed
0,Ushuaia,-54.8000,-68.3000,Clouds,broken clouds,279.15,276.47,279.15,279.15,991,87,10000,75,3.60
1,Thompson,55.7435,-97.8558,Clouds,scattered clouds,282.15,279.24,282.15,282.15,999,49,10000,40,5.66
2,Pathein,16.7833,94.7333,Clouds,broken clouds,301.88,304.64,301.88,301.88,1010,66,10000,65,2.73
3,Cape Town,-33.9258,18.4232,Rain,heavy intensity rain,290.04,290.08,289.82,290.37,1009,88,6000,90,6.17
4,Alice Springs,-23.7000,133.8833,Clear,clear sky,294.15,293.04,294.15,294.15,1024,28,10000,0,7.20
5,Williams Lake,52.1415,-122.1445,Clouds,few clouds,288.15,286.10,288.15,288.15,1005,15,10000,20,5.14
6,Laas,46.6166,10.7002,Clouds,overcast clouds,283.13,283.13,281.48,284.82,926,71,203,100,0.88


In [26]:
#Export the city data into a .csv.Display the DataFrame
cities_df.to_csv('cities.csv', encoding='utf-8', index= False)

In [32]:
#Inspect the data and remove the cities where the humidity > 100%.
#Skip this step if there are no cities that have humidity > 100%.
cities_removed = cities_df[cities_df['Humidity'] < 100]
cities_removed

,City,LAT,LON,Weather,Desription,Temp,Temp Feels,Temp Min,Temp Max,Pressure,Humidity,Visibility,Clouds,Wind Speed
0,Ushuaia,-54.8000,-68.3000,Clouds,broken clouds,279.15,276.47,279.15,279.15,991,87,10000,75,3.60
1,Thompson,55.7435,-97.8558,Clouds,scattered clouds,282.15,279.24,282.15,282.15,999,49,10000,40,5.66
2,Pathein,16.7833,94.7333,Clouds,broken clouds,301.88,304.64,301.88,301.88,1010,66,10000,65,2.73
3,Cape Town,-33.9258,18.4232,Rain,heavy intensity rain,290.04,290.08,289.82,290.37,1009,88,6000,90,6.17
4,Alice Springs,-23.7000,133.8833,Clear,clear sky,294.15,293.04,294.15,294.15,1024,28,10000,0,7.20
5,Williams Lake,52.1415,-122.1445,Clouds,few clouds,288.15,286.10,288.15,288.15,1005,15,10000,20,5.14
6,Laas,46.6166,10.7002,Clouds,overcast clouds,283.13,283.13,281.48,284.82,926,71,203,100,0.88


In [ ]:
#  Get the indices of cities that have humidity over 100%.

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

In [ ]:
#plotting the data
#Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
# Save the plotted figures as .pngs.

In [ ]:
#Latitude vs. Temperature Plot


In [ ]:
#Latitude vs. Humidity Plot

In [ ]:
#Latitude vs. Cloudiness Plot

In [ ]:
#Latitude vs. Wind Speed Plot

In [ ]:
#Linear Regression
#  Northern Hemisphere - Max Temp vs. Latitude Linear Regression


In [ ]:
#Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression